In [28]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler



In [49]:
def create_feature(df: pd.DataFrame) -> None:
    df["air_process_diff"] = abs(df["Air temperature [K]"] - df["Process temperature [K]"])
    df["speed_power"] = df['Rotational speed [rpm]'] * (2 * np.pi / 60) / (df['Rotational speed [rpm]'] * (2 * np.pi / 60) * df['Torque [Nm]'])
    df['torque_power'] = df['Torque [Nm]'] / (df['Rotational speed [rpm]'] * (2 * np.pi / 60) * df['Torque [Nm]'])
    df["tool_process"] = df["Tool wear [min]"] * df["Process temperature [K]"]
    df["temp_ratio"] = df["Process temperature [K]"] / df["Air temperature [K]"]
    df["product_id_num"] = pd.to_numeric(df["Product ID"].str.slice(start=1))
    
    df.drop(columns="Product ID", inplace=True)
    try:
        df.drop(columns="id", inplace=True)
    except:
        df.drop(columns="UDI", inplace=True)
    
def identify_column_types(df: pd.DataFrame) -> tuple[list, list, list]:
    num_cols = df.select_dtypes("float").columns.tolist()
    int_cols = df.select_dtypes("integer").columns.tolist()
    cat_cols = df.select_dtypes("object").columns.tolist()
    
    return num_cols, int_cols, cat_cols

def load_data(path):
    
    df = pd.read_csv(path)
    create_feature(df)
    return df
    

In [50]:
train_path = "../data/raw/train.csv"
test_path = "../data/raw/test.csv"
origin_path = "../data/raw/machine failure.csv"

In [51]:
target_col = 'Machine failure'

num_cols = [
    'Air temperature [K]',
    'Process temperature [K]',
    'Rotational speed [rpm]',
    'Torque [Nm]',
    'Tool wear [min]'
]

binary_cols = [
    'TWF',
    'HDF',
    'PWF',
    'OSF',
    'RNF'
]

# cat_cols = 'Type'
cat_cols = ['Type']

In [45]:
df_train = load_data(train_path)
df_test = load_data(test_path)
df_origin = load_data(origin_path)

In [52]:
df = pd.concat([df_train, df_origin], axis=0)



In [53]:
le = LabelEncoder()
for col in cat_cols:
    df["encoded_" + col] = le.fit_transform(df[col])
    
df.drop(cat_cols, axis=1, inplace=True)

In [57]:
import xgboost as xgb
import optuna

In [62]:
import re
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split

X, y = df.drop("Machine failure", axis=1), df["Machine failure"]
X.columns = [re.sub(r"[^a-zA-Z0-9_]+", "_", col) for col in X.columns]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)


In [67]:
dtrain = xgb.DMatrix(data=X, label=y)
param = {"objective": "binary:logistic"}
num_round = 10
res = xgb.cv(
    param,
    dtrain,
    num_round,
    nfold=5,
    metrics={"error"},
    seed=42,
    callbacks=[xgb.callback.EvaluationMonitor(show_stdv=True)],
    early_stopping_rounds=30)

[0]	train-error:0.00362+0.00009	test-error:0.00367+0.00033
[1]	train-error:0.00362+0.00009	test-error:0.00367+0.00033
[2]	train-error:0.00362+0.00009	test-error:0.00367+0.00033
[3]	train-error:0.00362+0.00009	test-error:0.00367+0.00033
[4]	train-error:0.00361+0.00009	test-error:0.00365+0.00033
[5]	train-error:0.00361+0.00009	test-error:0.00365+0.00033
[6]	train-error:0.00361+0.00009	test-error:0.00364+0.00034
[7]	train-error:0.00361+0.00009	test-error:0.00364+0.00033
[8]	train-error:0.00361+0.00008	test-error:0.00365+0.00034
[9]	train-error:0.00360+0.00008	test-error:0.00365+0.00035


In [68]:
res

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.003625,0.000091,0.003674,0.000327
1,0.003625,0.000091,0.003674,0.000327
2,0.003625,0.000091,0.003674,0.000327
3,0.003621,0.000088,0.003667,0.000332
4,0.003614,0.000087,0.003654,0.000332
5,0.003613,0.000086,0.003654,0.000332
6,0.003614,0.000087,0.003640,0.000343
7,0.003609,0.000091,0.003640,0.000331
8,0.003606,0.000083,0.003647,0.000344
9,0.003597,0.000080,0.003647,0.000350


In [71]:
xgb_clf = xgb.XGBClassifier(random_state=42)

scores = cross_validate(
    xgb_clf, X, y, cv=skf, scoring = "roc_auc", n_jobs=-1
)

In [73]:
base_roc = scores["test_score"].mean()
print(f"Base ROCAUC: {base_roc:.5f}")

Base ROCAUC: 0.96686


In [92]:
import mlflow

In [93]:
def get_or_create_experiment(experiment_name):
    """
    Retrieve the ID of an existing MLflow experiment or create a new one if it doesn't exist.

    This function checks if an experiment with the given name exists within MLflow.
    If it does, the function returns its ID. If not, it creates a new experiment
    with the provided name and returns its ID.

    Parameters:
    - experiment_name (str): Name of the MLflow experiment.

    Returns:
    - str: ID of the existing or newly created MLflow experiment.
    """

    if experiment := mlflow.get_experiment_by_name(experiment_name):
        return experiment.experiment_id
    else:
        return mlflow.create_experiment(experiment_name)

In [94]:
experiment_id = get_or_create_experiment("Test Experiment")

In [95]:
experiment_id

'747810045357265472'

In [96]:
# Set the current active MLflow experiment
mlflow.set_experiment(experiment_id=experiment_id)

<Experiment: artifact_location='file:///home/moon/project/mlops-pm/notebooks/mlruns/747810045357265472', creation_time=1717573153708, experiment_id='747810045357265472', last_update_time=1717573153708, lifecycle_stage='active', name='Test Experiment', tags={}>

In [80]:
def objective(trial, X, y, cv, scoring):
    
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1100),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        
    }
    
    # Perform CV
    xgb_clf = xgb.XGBClassifier(**params, random_state=42, eval_metric="auc", objective="binary:logistic")
    scores = cross_validate(xgb_clf, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    # Compute ROC
    roc = scores["test_score"].mean()

    return roc

In [97]:
def champion_callback(study, frozen_trial):
    """
    Logging callback that will report when a new trial iteration improves upon existing
    best trial values.

    Note: This callback is not intended for use in distributed computing systems such as Spark
    or Ray due to the micro-batch iterative implementation for distributing trials to a cluster's
    workers or agents.
    The race conditions with file system state management for distributed trials will render
    inconsistent values with this callback.
    """

    winner = study.user_attrs.get("winner", None)

    if study.best_value and winner != study.best_value:
        study.set_user_attr("winner", study.best_value)
        if winner:
            improvement_percent = (abs(winner - study.best_value) / study.best_value) * 100
            print(
                f"Trial {frozen_trial.number} achieved value: {frozen_trial.value} with "
                f"{improvement_percent: .4f}% improvement"
            )
        else:
            print(f"Initial trial {frozen_trial.number} achieved value: {frozen_trial.value}")

In [99]:
%%time
with mlflow.start_run(experiment_id=experiment_id, run_name="first_attempt", nested=True):
    # Create study that minimizes
    study = optuna.create_study(direction="maximize")
    
    # Wrap the objective inside a lambda with the relevant arguments
    # Pass additional arguments inside another function
    func = lambda trial: objective(trial, X, y, cv=skf, scoring="roc_auc")

    # Start optimizing with 100 trials
    study.optimize(func, n_trials=10, callbacks=[champion_callback])
    
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_roc", study.best_value)
    
    # Log tags
    mlflow.set_tags(
        tags={
            "project": "PM Project",
            "optimizer_engine": "optuna",
            "model_family": "xgboost",
            "feature_set_version": 1,
        }
    )

    # # Log a fit model instance
    # model = xgb.XGBClassifier(study.best_params, random_state=42, eval_metric="auc", objective="binary:logistic")
    # model.fit(X, y)
    

[I 2024-06-05 16:45:52,232] A new study created in memory with name: no-name-fda3d10c-5cbf-4310-9088-c68543034d47
[I 2024-06-05 16:46:08,784] Trial 0 finished with value: 0.9693962049945146 and parameters: {'n_estimators': 543, 'learning_rate': 0.002498568902488001, 'max_depth': 13, 'subsample': 0.7640918227808493, 'colsample_bytree': 0.5687143230169226}. Best is trial 0 with value: 0.9693962049945146.


Initial trial 0 achieved value: 0.9693962049945146


[I 2024-06-05 16:46:22,916] Trial 1 finished with value: 0.9635169471857153 and parameters: {'n_estimators': 606, 'learning_rate': 0.21100361555467928, 'max_depth': 6, 'subsample': 0.6952270899138203, 'colsample_bytree': 0.7065009235053148}. Best is trial 0 with value: 0.9693962049945146.
[I 2024-06-05 16:46:34,788] Trial 2 finished with value: 0.9574487885890948 and parameters: {'n_estimators': 923, 'learning_rate': 0.0013369847082727053, 'max_depth': 5, 'subsample': 0.8314654915721402, 'colsample_bytree': 0.9641483506116014}. Best is trial 0 with value: 0.9693962049945146.
[I 2024-06-05 16:46:51,739] Trial 3 finished with value: 0.9712582044070028 and parameters: {'n_estimators': 645, 'learning_rate': 0.010264632795934264, 'max_depth': 10, 'subsample': 0.9956769478086567, 'colsample_bytree': 0.5780791407093516}. Best is trial 3 with value: 0.9712582044070028.


Trial 3 achieved value: 0.9712582044070028 with  0.1917% improvement


[I 2024-06-05 16:47:12,193] Trial 4 finished with value: 0.9687272341005864 and parameters: {'n_estimators': 1044, 'learning_rate': 0.0017949884934022559, 'max_depth': 9, 'subsample': 0.764114496600007, 'colsample_bytree': 0.5640395581937845}. Best is trial 3 with value: 0.9712582044070028.
[I 2024-06-05 16:47:32,799] Trial 5 finished with value: 0.963050971377782 and parameters: {'n_estimators': 734, 'learning_rate': 0.24582459107393925, 'max_depth': 12, 'subsample': 0.6926321601393719, 'colsample_bytree': 0.7537352874779284}. Best is trial 3 with value: 0.9712582044070028.
[I 2024-06-05 16:47:43,747] Trial 6 finished with value: 0.9679102930874917 and parameters: {'n_estimators': 1012, 'learning_rate': 0.010342507533977413, 'max_depth': 3, 'subsample': 0.7973770536767587, 'colsample_bytree': 0.795649213432255}. Best is trial 3 with value: 0.9712582044070028.
[I 2024-06-05 16:47:47,640] Trial 7 finished with value: 0.956782448897407 and parameters: {'n_estimators': 272, 'learning_rate

CPU times: user 2.51 s, sys: 1.19 s, total: 3.7 s
Wall time: 2min 29s


[I 2024-06-05 16:29:22,810] A new study created in memory with name: no-name-95f7cafb-3d6b-4588-ba31-47e182bae1fb
[I 2024-06-05 16:29:32,903] Trial 0 finished with value: 0.9652396862289561 and parameters: {'n_estimators': 617, 'learning_rate': 0.001399771218293194, 'max_depth': 5, 'subsample': 0.5392642859683296, 'colsample_bytree': 0.5448109404935677}. Best is trial 0 with value: 0.9652396862289561.
[I 2024-06-05 16:29:37,592] Trial 1 finished with value: 0.9586865578546299 and parameters: {'n_estimators': 230, 'learning_rate': 0.0012947107883542277, 'max_depth': 6, 'subsample': 0.7140267115610769, 'colsample_bytree': 0.8811867895750225}. Best is trial 0 with value: 0.9652396862289561.
[I 2024-06-05 16:30:00,890] Trial 2 finished with value: 0.9680068354092315 and parameters: {'n_estimators': 651, 'learning_rate': 0.009250240924955157, 'max_depth': 13, 'subsample': 0.9692860161962089, 'colsample_bytree': 0.9972574494334845}. Best is trial 2 with value: 0.9680068354092315.
[I 2024-06-

CPU times: user 2.18 s, sys: 1.31 s, total: 3.49 s
Wall time: 2min 21s


In [82]:
print(f"Base ROCAUC: {base_roc:.5f}")
print(f"Optimized ROCAUC: {study.best_value:.5f}")

Base ROCAUC: 0.96686
Optimized ROCAUC: 0.97082


In [83]:
print("Best params:")
for key, value in study.best_params.items():
    print(f"\t{key}: {value}")

Best params:
	n_estimators: 959
	learning_rate: 0.01253278960281304
	max_depth: 7
	subsample: 0.7544060353752393
	colsample_bytree: 0.9822523112702799


In [86]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.